# Cargando un archivos de datos a un DataFrame

In [2]:
#importar módulos
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#crear sesión para poder acceder a todas las API de Spark
spark = SparkSession \
    .builder \
    .appName("Introducción a Spark DataFrame") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#define el data schema del archivo que queremos leer
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    

# lee el archivo csv (con el data schema definido) en un dataframe de Spark. Usa el delimitador "tab" 

purchaseDataframe = spark.read.csv(
    "/FileStore/tables/purchases.csv", 
    header=True, schema=purchaseSchema, sep="\t")

#Muestra 3 filas de nuestro Dataframe
purchaseDataframe.show(4)

+----------+-----+----------+----------------+------+--------+
 Date| Time| City| Item| Total| Payment|
+----------+-----+----------+----------------+------+--------+
2012-01-01|09:00| San Jose| Men's Clothing|214.05| Amex|
2012-01-01|09:00|Fort Worth|Women's Clothing|153.57| Visa|
2012-01-01|09:00| San Diego| Music| 66.08| Cash|
2012-01-01|09:00|Pittsburgh| Pet Supplies|493.51|Discover|
+----------+-----+----------+----------------+------+--------+
only showing top 4 rows

## Contamos el número de filas, luego imprimimos el dataframe schema y las estadísticas de nuestra data.

In [4]:
#Cuenta el número de filas de nuestra dataframe
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)

#Muestra el dataframe schema
purchaseDataframe.printSchema()

#Muestra estadísticas del campo que elegimos "Total" 
purchaseDataframe.describe('Total').show()

number of rows: 100
root
-- Date: date (nullable = true)
-- Time: string (nullable = true)
-- City: string (nullable = true)
-- Item: string (nullable = true)
-- Total: float (nullable = true)
-- Payment: string (nullable = true)

+-------+------------------+
summary| Total|
+-------+------------------+
 count| 100|
 mean|232.05479980945586|
 stddev|147.53289232427304|
 min| 2.0|
 max| 493.51|
+-------+------------------+

## Creamos un nuevo dataframe como subconjunto del dataframe que cargamos inicialmente

In [6]:
#Crea un nuevo dataframe con las columnas "City" y "Total"
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])
newDataframe.show(5); #Muestra las 3 primeras filas
newDataframe.printSchema() #Muestra el dataset schema del nuuevo dataframe

+----------+------+
 City| Total|
+----------+------+
 San Jose|214.05|
Fort Worth|153.57|
 San Diego| 66.08|
Pittsburgh|493.51|
 Omaha|235.63|
+----------+------+
only showing top 5 rows

root
-- City: string (nullable = true)
-- Total: float (nullable = true)

## Sumando un valor constante a todas las filas de una columna

In [8]:
#Sumando un valor constante de 10 a todas las filas de la columna "Total"
purchaseDataframe.select(purchaseDataframe['City'],purchaseDataframe['Total'],
                         purchaseDataframe['Total']+10).show(5)

+----------+------+------------+
 City| Total|(Total + 10)|
+----------+------+------------+
 San Jose|214.05| 224.05|
Fort Worth|153.57| 163.57|
 San Diego| 66.08| 76.08|
Pittsburgh|493.51| 503.51|
 Omaha|235.63| 245.63|
+----------+------+------------+
only showing top 5 rows

## Uso de filtros en un dataframe usando una condición definida

In [10]:
#Filtra solo las filas cuyo valor de la columna "Total" sea mayor a 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(5)

+----------+-----+----------+-------------------+------+----------+
 Date| Time| City| Item| Total| Payment|
+----------+-----+----------+-------------------+------+----------+
2012-01-01|09:00| San Jose| Men's Clothing|214.05| Amex|
2012-01-01|09:00|Pittsburgh| Pet Supplies|493.51| Discover|
2012-01-01|09:00| Omaha|Children's Clothing|235.63|MasterCard|
2012-01-01|09:00| Stockton| Men's Clothing|247.18|MasterCard|
2012-01-01|09:00| Austin| Cameras| 379.6| Visa|
+----------+-----+----------+-------------------+------+----------+
only showing top 5 rows

## Ordenando un dataframe por una columna definida

In [12]:
#Ordena el dataframe por la columna "City"
sortedByCity = purchaseDataframe.orderBy('City').show(5)
sortedByCity = purchaseDataframe.orderBy('City', 'Total').show(100)

+----------+-----+-----------+------------+------+----------+
 Date| Time| City| Item| Total| Payment|
+----------+-----+-----------+------------+------+----------+
2012-01-01|09:03|Albuquerque|Pet Supplies| 440.7| Cash|
2012-01-01|09:01| Anchorage| DVDs| 6.38| Amex|
2012-01-01|09:01| Anchorage| Crafts| 22.36| Amex|
2012-01-01|09:01| Anchorage| Music|298.86|MasterCard|
2012-01-01|09:03| Anchorage| DVDs| 390.2| Visa|
+----------+-----+-----------+------------+------+----------+
only showing top 5 rows

+----------+-----+--------------+--------------------+------+----------+
 Date| Time| City| Item| Total| Payment|
+----------+-----+--------------+--------------------+------+----------+
2012-01-01|09:03| Albuquerque| Pet Supplies| 440.7| Cash|
2012-01-01|09:01| Anchorage| DVDs| 6.38| Amex|
2012-01-01|09:01| Anchorage| Crafts| 22.36| Amex|
2012-01-01|09:01| Anchorage| Music|298.86|MasterCard|
2012-01-01|09:04| Anchorage| Health and Beauty|368.42|MasterCard|
2012-01-01|09:03| Anchorage| DVDs| 390.2| Visa|
2012-01-01|09:02| Atlanta| Toys|254.62| Discover|
2012-01-01|09:01| Aurora|Consumer Electronics|117.81|MasterCard|
2012-01-01|09:03| Austin| Crafts| 127.8| Discover|
2012-01-01|09:01| Austin| Sporting Goods|327.75| Discover|
2012-01-01|09:00| Austin| Cameras| 379.6| Visa|
2012-01-01|09:00| Austin| Cameras|469.63|MasterCard|
2012-01-01|09:02| Boise| Baby|130.54| Discover|
2012-01-01|09:02| Boise| Video Games|350.55| Discover|
2012-01-01|09:00| Boston| Cameras|418.94| Amex|
2012-01-01|09:00| Buffalo| Women's Clothing|483.82| Visa|
2012-01-01|09:02| Chandler| Books|344.09| Discover|
2012-01-01|09:02| Chandler| Books|414.08| Cash|
2012-01-01|09:03| Charlotte| DVDs|440.11| Visa|
2012-01-01|09:03| Chicago| Garden| 16.15| Visa|
2012-01-01|09:01| Chicago| Books| 31.08| Cash|
2012-01-01|09:03| Cincinnati| DVDs|299.55| Visa|
2012-01-01|09:00|Corpus Christi| Toys| 25.38| Discover|
2012-01-01|09:02| Durham| Health and Beauty|131.97| Visa|
2012-01-01|09:02| Durham| Toys|425.79| Visa|
2012-01-01|09:01| Fort Wayne| Men's Clothing|370.55| Amex|
2012-01-01|09:00| Fort Worth| Women's Clothing|153.57| Visa|
2012-01-01|09:00| Fort Worth| Toys|213.88| Visa|
2012-01-01|09:01| Fremont| Baby|222.61| Cash|
2012-01-01|09:03| Fremont| Men's Clothing|243.05| Discover|
2012-01-01|09:02| Fresno| Crafts|196.83| Visa|
2012-01-01|09:01| Fresno| CDs|466.64|MasterCard|
2012-01-01|09:02| Garland| Garden|134.33| Discover|
2012-01-01|09:03| Greensboro| Garden| 11.96|MasterCard|
2012-01-01|09:02| Greensboro| Computers|140.94| Discover|
2012-01-01|09:00| Greensboro| DVDs|290.82|MasterCard|
2012-01-01|09:02| Honolulu| Cameras|345.18| Discover|
2012-01-01|09:00| Houston| Baby|309.16| Visa|
2012-01-01|09:02| Indianapolis| Books|135.96| Discover|
2012-01-01|09:03| Jacksonville| Computers|359.65|MasterCard|
2012-01-01|09:02| Kansas City| Garden|302.69|MasterCard|
2012-01-01|09:00| Las Vegas| Video Games| 53.26| Visa|
2012-01-01|09:00| Las Vegas| Books| 93.39| Visa|
2012-01-01|09:02| Lexington| Women's Clothing|359.29| Visa|
2012-01-01|09:00| Lincoln| Garden| 136.9| Visa|
2012-01-01|09:04| Lubbock| Sporting Goods|452.07| Visa|
2012-01-01|09:01| Madison| Men's Clothing| 16.78| Visa|
2012-01-01|09:03| Memphis| Toys| 9.47|MasterCard|
2012-01-01|09:03| Milwaukee| Men's Clothing|418.91| Discover|
2012-01-01|09:02| Minneapolis| Computers|182.05| Visa|
2012-01-01|09:03| Nashville| Men's Clothing|297.43| Amex|
2012-01-01|09:03| New Orleans| Books|157.08| Discover|
2012-01-01|09:04| New Orleans| DVDs|242.31| Discover|
2012-01-01|09:02| New York| Garden| 18.27| Discover|
2012-01-01|09:02| New York|Consumer Electronics|153.84| Amex|
2012-01-01|09:00| New York|Consumer Electronics| 296.8| Cash|
2012-01-01|09:00| Newark| Video Games| 39.75| Cash|
2012-01-01|09:02| Norfolk| Women's Clothing|189.01| Amex|
2012-01-01|09:04| Norfolk|Consumer Electronics|480.24| Cash|
2012-01-01|09:04| Oakland| Garden|155.33| Visa|
2012-01-01|09:00| Omaha| Children's Clothing|235.63|MasterCard|
2012-01-01|09:

## Calculando el número de transacciones por cada ciudad

In [14]:
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.show(10)

+------------+-----+
 City|count|
+------------+-----+
 Phoenix| 1|
 Omaha| 3|
 Anchorage| 5|
 Greensboro| 3|
 Oakland| 1|
 San Antonio| 1|
Philadelphia| 2|
 Chandler| 2|
Indianapolis| 1|
 San Diego| 1|
+------------+-----+
only showing top 10 rows

## Creación de ID único e incremental en un dataframe
Podemos acceder por fila usando la función ".filter ()" a nuestra columna "ID incremental".

In [16]:
#importamos la función monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
    "index", monotonically_increasing_id())
newPurchasedDataframe.show(5)

row2Till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index']>=2) &
                                         (newPurchasedDataframe['index']<=4))
row2Till4.show()

+----------+-----+----------+-------------------+------+----------+-----+
 Date| Time| City| Item| Total| Payment|index|
+----------+-----+----------+-------------------+------+----------+-----+
2012-01-01|09:00| San Jose| Men's Clothing|214.05| Amex| 0|
2012-01-01|09:00|Fort Worth| Women's Clothing|153.57| Visa| 1|
2012-01-01|09:00| San Diego| Music| 66.08| Cash| 2|
2012-01-01|09:00|Pittsburgh| Pet Supplies|493.51| Discover| 3|
2012-01-01|09:00| Omaha|Children's Clothing|235.63|MasterCard| 4|
+----------+-----+----------+-------------------+------+----------+-----+
only showing top 5 rows

+----------+-----+----------+-------------------+------+----------+-----+
 Date| Time| City| Item| Total| Payment|index|
+----------+-----+----------+-------------------+------+----------+-----+
2012-01-01|09:00| San Diego| Music| 66.08| Cash| 2|
2012-01-01|09:00|Pittsburgh| Pet Supplies|493.51| Discover| 3|
2012-01-01|09:00| Omaha|Children's Clothing|235.63|MasterCard| 4|
+----------+-----+----------+-------------------+------+----------+-----+

Luego, para acceder a él por fila y columna, use la función ".select ()" que hemos usado anteriormente.

In [18]:
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index']==4).select('Total')
dataRow2ColumnTotal.show()

+------+
 Total|
+------+
235.63|
+------+

# Usando consultas SQL en un dataFrame

Crear un nuevo dataFrame a partir del subconjunto de nuestro dataFrame existente

In [20]:
#Vamos a crear una vista temporal a partir de nuestro dataframe inicial
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#Selecciona las columnas "Total" y "Payment" de la vista recientemente creada
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(5)



+------+----------+
 Total| Payment|
+------+----------+
214.05| Amex|
153.57| Visa|
 66.08| Cash|
493.51| Discover|
235.63|MasterCard|
+------+----------+
only showing top 5 rows

Ordenando los datos a partir de una columna definida

In [22]:
#Ordenando los datos por la columna "City"
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City")
orderByCity.show(10)

+----------+-----+-----------+--------------------+------+----------+
 Date| Time| City| Item| Total| Payment|
+----------+-----+-----------+--------------------+------+----------+
2012-01-01|09:03|Albuquerque| Pet Supplies| 440.7| Cash|
2012-01-01|09:01| Anchorage| Music|298.86|MasterCard|
2012-01-01|09:04| Anchorage| Health and Beauty|368.42|MasterCard|
2012-01-01|09:01| Anchorage| DVDs| 6.38| Amex|
2012-01-01|09:01| Anchorage| Crafts| 22.36| Amex|
2012-01-01|09:03| Anchorage| DVDs| 390.2| Visa|
2012-01-01|09:02| Atlanta| Toys|254.62| Discover|
2012-01-01|09:01| Aurora|Consumer Electronics|117.81|MasterCard|
2012-01-01|09:00| Austin| Cameras|469.63|MasterCard|
2012-01-01|09:00| Austin| Cameras| 379.6| Visa|
+----------+-----+-----------+--------------------+------+----------+
only showing top 10 rows

Filtraremos la columna "Total" para valores mayores a 200 y lo ordenaremos por la columna "Payment"

In [24]:
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE Total>200 ORDER BY Payment")
filterAndSortWithSQL.show(10)

+----------+-----+--------------+--------------------+------+-------+
 Date| Time| City| Item| Total|Payment|
+----------+-----+--------------+--------------------+------+-------+
2012-01-01|09:00|Virginia Beach| Children's Clothing|376.11| Amex|
2012-01-01|09:00| San Jose| Men's Clothing|214.05| Amex|
2012-01-01|09:03| Nashville| Men's Clothing|297.43| Amex|
2012-01-01|09:04| Philadelphia| Pet Supplies|332.48| Amex|
2012-01-01|09:01| Fort Wayne| Men's Clothing|370.55| Amex|
2012-01-01|09:00| Boston| Cameras|418.94| Amex|
2012-01-01|09:01| Pittsburgh| Sporting Goods|475.26| Amex|
2012-01-01|09:00| San Jose| Women's Clothing|215.82| Cash|
2012-01-01|09:00| New York|Consumer Electronics| 296.8| Cash|
2012-01-01|09:01| Riverside|Consumer Electronics|252.88| Cash|
+----------+-----+--------------+--------------------+------+-------+
only showing top 10 rows